In [ ]:
# imports

In [ ]:
def BookRecommenderCollaborativeSetup(ISBNandTitle, DataSample, ComparisonValue):
    '''
    A memory based collaborative recommender that takes a sample of book data, a comparison value, and a certain book and returns a dataframe where the books are sorted from most to least correlated to the selected book.

    Args:
        ISBNandTitle (string) : the combined ISBN and Title of a book, formatted like '0345337662 Interview with the Vampire'
        DataSample (dataframe) : the dataframe sample that is being used to retrieve book data.
        ComparisonValue (string) : the numeric column that is being used to compare books.

    Returns:
        a dataframe where the books are sorted from most to least correlated to the selected book.
        
    '''
	# start with the sample data. code that outside of the function and use it in here.
	
	# creating a table showing the ratings of each user in the sample data compared with each book in the sample data
    UserRatingsTable = DataSample.pivot_table(index='User-ID', columns='ISBN-and-Title', values=ComparisonValue)

	# getting the ratings for the selected book
    SelectedRatings = UserRatingsTable[ISBNandTitle]
	
	# getting the correlation of the other books with the selected book
    BooksLikeSelected = UserRatingsTable.corrwith(SelectedRatings)
	
	# making a dataframe of these correlations
    CorrelatedBooks = pd.DataFrame(BooksLikeSelected, columns=['Correlation'])
    CorrelatedBooks.dropna(inplace=True)
	
	# ensuring the book itself doesn't show up as a recommendation
	# if the book is recommended, remove it from the options
    if ISBNandTitle in CorrelatedBooks.index:
        CorrelatedBooks.drop(index=ISBNandTitle,inplace=True)
	
	# sorting by highest corrrelation 
    return CorrelatedBooks.sort_values('Correlation',ascending=False)

In [ ]:
def BookRecommenderCollaborativeMemScore(ISBNandTitle, DataSample):
	'''
	Takes the memory based collaborative recommender defined above and uses a preset ComparisonValue and .head(10) to return the top 10 most correlated books according to the 'Score' column.

	Args:
        ISBNandTitle (string) : the combined ISBN and Title of a book, formatted like '0345337662 Interview with the Vampire'
        DataSample (dataframe) : the dataframe sample that is being used to retrieve book data.
		
	Returns:
	a dataframe of the top 10 most correlated books according to the 'Score' column
	
	'''
	return BookRecommenderCollaborativeSetup(ISBNandTitle, DataSample, ComparisonValue='Score').head(10)
	

In [ ]:
# using score makes a lot of the books be seen as 'exactly correlated', so I will try with the actual rating also
def BookRecommenderCollaborativeMemRating(ISBNandTitle, DataSample):
	'''
	Takes the memory based collaborative recommender defined above and uses a preset ComparisonValue and .head(10) to return the top 10 most correlated books according to the 'Book-Rating' column.

	Args:
        ISBNandTitle (string) : the combined ISBN and Title of a book, formatted like '0345337662 Interview with the Vampire'
        DataSample (dataframe) : the dataframe sample that is being used to retrieve book data.
		
	Returns:
	a dataframe of the top 10 most correlated books according to the 'Book-Rating' column
	
	'''
	return BookRecommenderCollaborativeSetup(ISBNandTitle, DataSample, ComparisonValue='Book-Rating').head(10)

In [ ]:
# making a function that shows the correlation of books
def BookRecommenderContentSetup(ISBNandTitle, CosineSimilarity=CosineSim):
    '''
    A content based recommender that takes an array of book similarities based on determined factors and a certain book and returns a dataframe where the books are sorted from most to least correlated to the selected book.

    Args:
        ISBNandTitle (string) : the combined ISBN and Title of a book, formatted like '0345337662 Interview with the Vampire'
        CosineSimilarity (array) : the similarity of each book to each other based on determined factors.

    Returns:
        a dataframe where the books are sorted from most to least correlated to the selected book according to determined factors.

    '''
    # getting the index of the box matching with the input
    BookIndex = Indices[ISBNandTitle]

    # getting the similarity scores of all books compared to that book
    SimScores = list(enumerate(CosineSimilarity[BookIndex]))

    # making into a dataframe the similarity scores into a dataframe
    BooksSimScores = pd.DataFrame(SimScores, columns=['Index-Value', 'Similarity-Score'])

    # adding in my ISBN title column for clarity
    BooksSimScoresDetails = BooksSimScores.merge(RatedBooksSample350['ISBN-and-Title'], how='left', left_on='Index-Value', right_index=True)
    BooksSimScoresDetails

    # dropping the Index-Value column
    BooksSimScoresDetails.drop(columns='Index-Value',inplace=True)

    # rearranging columns
    BooksSimScoresDetails = BooksSimScoresDetails[['ISBN-and-Title', 'Similarity-Score']]

    # ensuring the book itself doesn't show up as a recommendation
	# if the book is recommended, remove it from the options
    if ISBNandTitle in BooksSimScoresDetails['ISBN-and-Title'].values:

        # finding the index where ISBNandTitle exists
        IndexToDrop = BooksSimScoresDetails[BooksSimScoresDetails['ISBN-and-Title'] == ISBNandTitle].index

        # dropping any rows with ISBNandTitle
        BooksSimScoresDetails.drop(index=IndexToDrop, inplace=True)

    # sorting by similarity score
    return BooksSimScoresDetails.sort_values('Similarity-Score',ascending=False)

In [ ]:
# showing just the top 10
def BookRecommenderContent(ISBNandTitle, CosineSimilarity=CosineSim):
    ''' 
	Takes the content based collaborative recommender defined above and returns the top 10 most correlated books according to determined factors.

    Args:
        ISBNandTitle (string) : the combined ISBN and Title of a book, formatted like '0345337662 Interview with the Vampire'
        CosineSimilarity (array) : the similarity of each book to each other based on determined factors.

    Returns:
        a dataframe of the top 10 most correlated books sorted from most to least correlated to the selected book according to determined factors.
        
    '''
    return BookRecommenderContentSetup(ISBNandTitle, CosineSimilarity=CosineSim).head(10)

In [ ]:
def CombiningRecommenders(ISBNandTitle, DataSample):
    '''
    Combining the collaborative memory based recommenders and the content based recommender defined above by taking the scores, averaging them, and re-sorting according to the new score.

    Args:
        ISBNandTitle (string) : the combined ISBN and Title of a book, formatted like '0345337662 Interview with the Vampire'
        DataSample (dataframe) : the dataframe sample that is being used to retrieve book data.
    
    Returns:
        a dataframe of the top 10 most correlated books sorted from most to least correlated to the selected book according to the combined average of the 3 recommenders defined above.
    
    '''
    # collaborative score
    ScoresCollaborative = BookRecommenderCollaborativeSetup(ISBNandTitle, DataSample, 'Score')
    
    # collaborative rating
    RatingsCollaborative = BookRecommenderCollaborativeSetup(ISBNandTitle, DataSample, 'Book-Rating')

    # content based
    ContentBased = BookRecommenderContentSetup(ISBNandTitle)
    

    # combining and averaging

    # putting together in a dataframe
    CollaborativeJoin = pd.merge(ScoresCollaborative, RatingsCollaborative, on='ISBN-and-Title')
    AllScoresJoined = pd.merge(CollaborativeJoin,ContentBased, on='ISBN-and-Title')

    # renaming columns
    AllScoresJoined.rename(columns={'Correlation_x': 'Score-Similarity', 'Correlation_y':'Rating-Similarity', 'Similarity-Score':'Cosine-Similarity'}, inplace=True)


    # combining all 3 recommender scores into an average to account for all factors
    AllScoresJoined['Score-Rating-Cosine'] = (AllScoresJoined['Score-Similarity'] + AllScoresJoined['Rating-Similarity'] + AllScoresJoined['Cosine-Similarity']) / 3



    return AllScoresJoined.sort_values('Score-Rating-Cosine',ascending=False).head(10)
    # re-sort